# 🛡️ SENTINEL HUD - ENTRENAMIENTO UNIVERSAL (Local/Colab)

Este notebook está diseñado para funcionar tanto en **Google Colab** como en tu **PC Local**.
Detecta automáticamente el entorno y ajusta los comandos para el entrenamiento académico.

In [ ]:
import os
import sys
import glob
from pathlib import Path

# Detectar entorno
IN_COLAB = 'google.colab' in sys.modules

print(f"🔹 Entorno detectado: {'GOOGLE COLAB' if IN_COLAB else 'LOCAL PC'}")

## 1. Configuración del Entorno
Clonamos YOLOv5 e instalamos dependencias.

In [ ]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5

%cd yolov5
!pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output, display

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

## 2. Dataset
Si estás en Colab, descomprime el zip. Si estás en local, usa la carpeta `dataset_final` directamente.

In [ ]:
if IN_COLAB:
    if os.path.exists('../dataset_final.zip'):
        !unzip -q -o ../dataset_final.zip -d ../dataset_final
        print("✅ Dataset descomprimido en Colab.")
    else:
        print("⚠️ ATENCIÓN: Sube 'dataset_final.zip' a la raiz de Colab.")
else:
    print("✅ Usando dataset local en '../dataset_final'.")

## 3. Entrenamiento
Entrenamos YOLOv5s.

**Configuración Automática de Epochs**:
- En **Local (CPU)**: 10 epochs (rápido, ~5-10 min) para generar gráficas académicas.
- En **Colab (GPU)**: 100 epochs para entrenamiento real.

In [ ]:
if os.path.exists('../dataset_final/custom_data.yaml'):
    data_path = '../dataset_final/custom_data.yaml'
else:
    data_path = 'custom_data.yaml'

epochs = 100 if IN_COLAB else 10
print(f"🚀 Iniciando entrenamiento con {epochs} epochs en {'GPU' if IN_COLAB else 'CPU'}...")

!python train.py --img 640 --batch 16 --epochs {epochs} --data {data_path} --weights yolov5s.pt --name sentinel_exp --cache

## 4. Resultados Automáticos
A diferencia de un script estático, este bloque busca automáticamente la **última carpeta de entrenamiento** para evitar errores de 'FileNotFoundError'.

In [ ]:
# Buscar la carpeta sentinel_exp más reciente
folders = glob.glob('runs/train/sentinel_exp*')
if folders:
    latest_folder = max(folders, key=os.path.getctime)
    print(f"📁 Cargando evidencias desde: {latest_folder}")
    
    for file in ['results.png', 'confusion_matrix.png', 'F1_curve.png']:
        img_path = os.path.join(latest_folder, file)
        if os.path.exists(img_path):
            print(f"✅ Mostrando {file}")
            display(Image(filename=img_path, width=800))
        else:
            print(f"❌ {file} no generado aún.")
else:
    print("❌ No se encontró ninguna carpeta de entrenamiento 'sentinel_exp'.")

## 5. Exportar Modelo

In [ ]:
folders = glob.glob('runs/train/sentinel_exp*')
if folders:
    latest_folder = max(folders, key=os.path.getctime)
    best_model = os.path.join(latest_folder, 'weights/best.pt')
    
    if IN_COLAB:
        from google.colab import files
        files.download(best_model)
    else:
        print(f"✅ Entrenamiento finalizado. Tu mejor modelo está en: {os.path.abspath(best_model)}")